In [ ]:
import sys

orig_stdout = sys.stdout
f = open('out.txt', 'w')
sys.stdout = f

sys.stdout = orig_stdout

In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [ ]:
def combine_dict(list_of_dictionaries):
    dic = defaultdict(lambda: list())
    for d in list_of_dictionaries:  
        for item in d.items():
            dic[item[0]].append(item[1])
    return dic
    
def scale(df,col,on_range = True):
    if on_range: # Scaling based on Range
        df[col] -= df[col].min()
        df[col] /= df[col].max()
    else: # Scaling based on SD
        sd = df[col].std()
        df[col] -= df[col].mean()
        df[col] /= sd
    return

In [ ]:
fil = pd.read_excel('./Course_Advisory_Data.xlsx')
pre_req = pd.read_excel('./Prerequisites of All Courses.xlsx')
pre_req.set_index('Code',inplace=True)
pre_req.drop(['Sr#','Type','Status','Equivalence'],axis=1,inplace=True)

In [ ]:
cols = fil.columns.to_list()
semesters = list(fil.Semester.unique())
roll_nos = list(fil['Sr. No'].unique())
cols

In [ ]:
# Dictionary for Course Code and Course Title and Credit Hours
course_guide = combine_dict([dict(zip(fil['Course Code'], fil['Course Title'])),dict(zip(fil['Course Code'], fil['Credit Hours']))])
# crs = pre_req.index.to_list()
# for c in crs:
#     if c not in course_guide:
#         course_guide[c] = pre_req.ix[c].to_list()[0:2]

# Dictionary for Grade and Grade Point
grade_dictionary = dict(zip(fil['Grade'], fil['Grade Point']))

In [ ]:

data = fil[:]
data.set_index('Sr. No',inplace= True)
data = data.drop(labels = ['Course Title','Grade','Credit Hours'],axis = 1)

roll_nos = data.index.unique().to_list()
# scale(data, 'CGPA')
# scale(data, 'SGPA')
# scale(data, 'Grade Point')

modified_data = pd.DataFrame(columns=['Roll_No'],data=roll_nos)
modified_data.set_index(['Roll_No'],inplace=True)

cols = ['Course','Grade_Point','Semester','Warning','SGPA','CGPA']

for c in cols:
    modified_data[c] = np.empty((len(roll_nos), 0)).tolist()

In [ ]:
roll_nos.sort()
for roll in roll_nos:
    for semester in semesters:
        semester = str(semester)
        try:
            df = pd.DataFrame(data.ix[roll])
            df = df.groupby(['Semester']).get_group(semester)
            modified_data.ix[roll].Semester.append([semester])
            modified_data.ix[roll].Course.append(list(df['Course Code'].to_list()))
            modified_data.ix[roll].Grade_Point.append(list(df['Grade Point'].to_list()))
            modified_data.ix[roll].Warning.append(list([df['Warning'].to_list()[0]]))
            modified_data.ix[roll].SGPA.append(list([df['SGPA'].to_list()[0]]))
            modified_data.ix[roll].CGPA.append(list([df['CGPA'].to_list()[0]]))
        except KeyError:
            pass

In [ ]:
modified_data.to_excel('New_Data.xlsx')
modified_data.to_pickle('New_Data_pickle.xlsx')

In [41]:
data = pd.read_pickle('./New_Data_pickle.xlsx')
cols = data.columns.to_list()
students = data.index.to_list()

In [42]:

xyData = pd.DataFrame(columns=['Taken_Courses','Grades','CGPA','SGPA','Warning','Recommended'])

# xyData.append({'Taken_Courses':data.loc[17291].Course[0],'Recommended':data.loc[17291].Course[1]}, ignore_index=True)

students = data.index.to_list() # Enrolled Students

stud_given = data.loc[17291]

for student in students:
    student_data = data.loc[student]
    for i in range(len(data.loc[student].Semester)-1):
        xyData = xyData.append({'Taken_Courses':student_data.Course,
        'Grades':student_data.Grade_Point,
        'CGPA': student_data.CGPA,
        'SGPA':student_data.SGPA,
        'Warning':student_data.Warning,
        'Recommended':student_data.Course} ,
        ignore_index=True)

xyData.to_pickle('xydata.sav')

In [43]:
xyData

,Taken_Courses,Grades,CGPA,SGPA,Warning,Recommended
0,"[[EE182, CL101, CS101, SS111, SS101, MT101, SL...","[[4.0, 3.0, 1.67, 3.33, 3.67, 1.67, 3.67, 4.0,...","[[2.92], [2.85], [2.82], [2.81], [2.83], [2.85...","[[2.92], [2.78], [2.75], [2.77], [2.9], [2.96]...","[[0], [0], [0], [0], [0], [0], [0]]","[[EE182, CL101, CS101, SS111, SS101, MT101, SL..."
1,"[[EE182, CL101, CS101, SS111, SS101, MT101, SL...","[[4.0, 3.0, 1.67, 3.33, 3.67, 1.67, 3.67, 4.0,...","[[2.92], [2.85], [2.82], [2.81], [2.83], [2.85...","[[2.92], [2.78], [2.75], [2.77], [2.9], [2.96]...","[[0], [0], [0], [0], [0], [0], [0]]","[[EE182, CL101, CS101, SS111, SS101, MT101, SL..."
2,"[[EE182, CL101, CS101, SS111, SS101, MT101, SL...","[[4.0, 3.0, 1.67, 3.33, 3.67, 1.67, 3.67, 4.0,...","[[2.92], [2.85], [2.82], [2.81], [2.83], [2.85...","[[2.92], [2.78], [2.75], [2.77], [2.9], [2.96]...","[[0], [0], [0], [0], [0], [0], [0]]","[[EE182, CL101, CS101, SS111, SS101, MT101, SL..."
3,"[[EE182, CL101, CS101, SS111, SS101, MT101, SL...","[[4.0, 3.0, 1.67, 3.33, 3.67, 1.67, 3.67, 4.0,...","[[2.92], [2.85], [2.82], [2.81], [2.83], [2.85...","[[2.92], [2.78], [2.75], [2.77], [2.9], [2.96]...","[[0], [0], [0], [0], [0], [0], [0]]","[[EE182, CL101, CS101, SS111, SS101, MT101, SL..."
4,"[[EE182, CL101, CS101, SS111, SS101, MT101, SL...","[[4.0, 3.0, 1.67, 3.33, 3.67, 1.67, 3.67, 4.0,...","[[2.92], [2.85], [2.82], [2.81], [2.83], [2.85...","[[2.92], [2.78], [2.75], [2.77], [2.9], [2.96]...","[[0], [0], [0], [0], [0], [0], [0]]","[[EE182, CL101, CS101, SS111, SS101, MT101, SL..."
...,...,...,...,...,...,...
7070,"[[CS118, SS113, SS150, MT119, CL117, SL150, EE...","[[0.0, 3.33, 3.33, 0.0, 2.33, 2.0, 1.67, 1.0],...","[[1.59], [2.9], [2.9]]","[[1.59], [3.08], [0.0]]","[[0], [0], [0]]","[[CS118, SS113, SS150, MT119, CL117, SL150, EE..."
7071,"[[CS118, SS113, SS150, MT119, CL117, SL150, EE...","[[0.0, 3.33, 3.33, 0.0, 2.33, 2.0, 1.67, 1.0],...","[[1.59], [2.9], [2.9]]","[[1.59], [3.08], [0.0]]","[[0], [0], [0]]","[[CS118, SS113, SS150, MT119, CL117, SL150, EE..."
7072,"[[MG599, MG546, MG547, MG549], [MG528, MG593, ...","[[3.67, 2.33, 3.67, 2.67], [0.0, 2.0, 0.0], [0...","[[3.09], [2.05], [2.05]]","[[3.09], [0.67], [0.0]]","[[1], [1], [1]]","[[MG599, MG546, MG547, MG549], [MG528, MG593, ..."
7073,"[[MG599, MG546, MG547, MG549], [MG528, MG593, ...","[[3.67, 2.33, 3.67, 2.67], [0.0, 2.0, 0.0], [0...","[[3.09], [2.05], [2.05]]","[[3.09], [0.67], [0.0]]","[[1], [1], [1]]","[[MG599, MG546, MG547, MG549], [MG528, MG593, ..."


In [ ]:

from collections import Counter

class KNN:
	def __init__(self, k=1):
		self.k = k
    
	def train(self, students_data):
		self.data = students_data

	def predict(self, stud_given):
		pass

	def evaluate(self, stud_given, recomm_courses):
		pass
